<style>
    pre {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
</style>

<div style="display:flex; justify-content:space-around; align-items:center; background-color:#cccccc; padding:5px; border:2px solid #333333;">
    <a href="https://www.um.es/web/estudios/grados/ciencia-ingenieria-datos/" target="_blank">
    <img src="https://www.um.es/documents/1073494/42130150/LogosimboloUMU-positivo.png" alt="UMU" style="height:200px; width:auto;">
    <a href="https://estudios.upct.es/grado/5251/inicio" target="_blank">
    <img src="https://www.upct.es/contenido/universidad/galeria/identidad-2021/logos/logos-upct/marca-upct/marca-principal/horizontal/azul.png" alt="UPCT" style="height:145px; width:auto;">
</div>

# Asignatura: **Deep Learning**

## Titulación: **Grado en Ciencia e Ingeniería de Datos**

## Práctica 3: Transformers
### **Sesión 3/3: Transformer**

**Autores**: Antonio Martínez Sánchez, Juan Morales Sánchez, José Luís Sancho Gómez y Juan Antonio Botía Blaya

<div style="page-break-before: always;"></div>

### Contenidos
- [Requisitos](#requisitos)
- [Transformer](#transformer)
- [Embedding posicional](#embedp)
- [Ejercicios](#ejercicios)

### Requisitos
<a class='anchor' id='requisitos'></a>

Se trabajará con notebooks de [Jupyter](https://jupyter.org/install) con código Python empleando como intérprete la última versión de [Miniconda](https://docs.anaconda.com/miniconda/). Se requiere la preinstalación (se recomienda utilizar [pip](https://pypi.org/project/pip/)) de los siguientes paquetes de Python:

- [Numpy](https://pypi.org/project/numpy/) (computación numérica)
- [Tensorflow](https://www.tensorflow.org/install/pip?hl=es-419#linux) que incluye a Keras (deep learning)
- [Scikit-learn](https://pypi.org/project/scikit-learn/) (machine learning)
- [Matplotlib](https://pypi.org/project/matplotlib/) (visualización de datos)
- [Pandas](https://pypi.org/project/seaborn/) (manipulación de datos tabulados)

### Transformer
<a class='anchor' id='transformer'></a>

Las redes RNN son difíciles de entrenar, aun así, los LSTM han sido la arquitectura más utilizada para la traducción de texto hasta la llegada de los transformers allá por el 2017. Los transformers implementan de forma práctica el concepto de multi-atención. Durante el *embedding* hemos generado un espacio que captura la relación entre los tokens vecinos analizando una serie de secuencias. Sin embargo, este proceso no permite representar relaciones más complejas. Volviendo al caso del lenguaje natural, el significado de una palabra es específico de su contexto. Por ejemplo, la palabra "estación" tiene un significado diferente en las frases "el tren salió de la estación" o "el verano es la estación más calurosa del año". Otro ejemplo claro son los pronombres "él", "esto", "aquello", ..., que pueden variar su significado en una misma frase. Por esto un modelo debe aprender los diferentes contextos y saber cuándo aplicarlos a cada token. Los Transformes surgieron para conseguir este propósito, para ello utilizan el mecanismo de atencián asignando a cada instancia de un token una semántica en función de su contexto. Además, la multi-atención permite factorizar en espacio de características en diferentes subespacios, lo cual había mostrado un buen comportamiento con las redes CNN. 

En esta sesión vamos a implementar un encoder basado en un Tranformer, para ello construiremos una clase que herede de [Layer](https://keras.io/api/layers/base_layer/) de Keras. Tal y como viene explicado en la documentación de Keras, es necesario implementar el constructor dónde se inicialicen las variables de clase y el método *call()* que será ejecutado durante el entrenamiento y la inferencia. Además, también tendremos que implementar el método *get_config()* para almacenar la variables de clase durante la serialización, necesaria para almacenar el modelo en disco. El Transformer encoder a implementar se basará en siguiente esquema:

<img src="./figs/trans_encoder.png" width="400">

Keras tiene implementadas clases para las capas [MultiHeadAttention](https://keras.io/api/layers/attention_layers/multi_head_attention/), [LayerNormalization](https://keras.io/api/layers/normalization_layers/layer_normalization/) y [Dense](https://keras.io/api/layers/core_layers/dense/). La capa MultiHeadAttention requiere de tres entradas: query, key and values. Es importante resaltar que en este caso, puesto que se trata de un encoder, las tres entradas de la capa MultiHeadAttention se corresponden con la única entrada del encoder. También es importante remarcar que este capa recibirá la entrada de una capa de embedding que generará un máscara, esta máscara tiene que ser recogida por el método *call()* del transformer encoder implementado y transformada para que dimensionalmente encaje con las dimensiones requeridas por la capa MultiHeadAttention.  

### Embedding posicional
<a class='anchor' id='codep'></a>

Un transformer como el de la sección anterior por sí solo es insensible a la posición de los tokens en la secuencia, como el MLP. No obstante, cuando introdujimos el LSTM, ya comentamos que una de sus ventajas era que las redes RNN son sensibles al orden de la secuencia. Así que se necesita que la entrada contenga dos informaciones, la semántica de cada token en función de su relación con los tokens vecinos (embedding) y la posición del token en la secuencia. Para codificar la información sobre la posición podría bastar con crear un índice, pero en la práctica no es una buena idea porque a las redes neuronales no les gusta los números grandes o las distribuciones discretas. En el árticulo original dónde se presentaron los transformers [Attention Is All You Need](https://arxiv.org/abs/1706.03762) emplea una función trigonométrica para codificar la posición en el rango -1 y 1. Aquí emplearemos una aproximación más simple pero efectiva, crearemos un array con las posiciones de cada token y las embeberemos en un espacio vectorial, después los vectores embebidos de posición se sumarán a los vectores embebedidos de los tokens. Esta técnica, conocida como *embedding posicional*, se resume en el siguiente esquema:

<img src="./figs/pos_embedding.png" width="400">

La clase que implemente el esquema de arriba ha de implementar un método llamado *compute_mask* que será invocado por el framework. Como partiremos de una vectorización tipo *Integer* con secuencias de diferente tamaño, la máscara servirá para definir qué posiciones de la secuencia contienen AAs.

### Ejercicios
<a class='anchor' id='embedp'></a>

**E1:** Implementa la clase MyTransformerEncoder según el esquema descrito anteriormente.

**E2:** Construye un clasificador a partir de MyTransformerEncoder. Para ello aplica una capa de [Embedding](https://keras.io/api/layers/core_layers/embedding/) antes de pasar los datos a MyTransfomerEncoder, luego aplica un [GlobalMaxPooling1D](https://keras.io/api/layers/pooling_layers/global_max_pooling1d/) a la salida y una capa final de activación sigmoide.

**E3:** Carga el dataset de los péptidos y entrena el codificador completo del E2 y evalúa su precisión.

**E4:** Implementa la clase MyPositionEmbedding según el esquema descrito anteriormente.

**E5:** Construye un clasificador sustituyendo la capa de Embedding por la clase implementada en E4. Entrena y evalúa el nuevo modelo. ¿Qué nueva información se ha añadidio al clasificador? ¿ha mejorado los resultados? ¿por qué?

**E6:** ¿Por qué parece que la mejora de utilizar la información posicional no es muy significativa? ¿a qué se podría deber?

**E7:** Ajusta los hiperparámetros del modelo en E5 y el LSTM bidireccional del E4 de la Sesión 2 hasta obtener una configuración óptima para cada modelo. Utiliza el clúster [ATLAS](https://scc.atica.um.es/intro.html) para acelerar el entrenamiento y testeo de los modelos. ¿Qué modelo consigue un mejor funcionamiento? ¿qué puedes decir sobre el entrenamiento de ambos modelos? 

El embedding posicional puede no mejorar los resultados por varias razones:

- **Naturaleza de la tarea:** Si la tarea no depende fuertemente del orden de los elementos (por ejemplo, clasificaciones basadas en la presencia de ciertos tokens, más que en su secuencia), la información posicional puede aportar poco valor.

- **Longitud y complejidad de las secuencias:** En secuencias muy cortas o cuando la relación de orden no es crucial, la ventaja de conocer la posición es mínima.

- **Diseño del modelo:** Es posible que otros componentes del modelo (como capas de atención) ya capten relaciones contextuales de manera efectiva, haciendo redundante el aporte del embedding posicional.

- **Implementación o hiperparámetros:** Si el método para incorporar la posición (por ejemplo, la forma de combinar el embedding posicional con el de tokens) no se ajusta bien o si los hiperparámetros no están optimizados, el beneficio puede ser nulo o incluso contraproducente.

En resumen, el embedding posicional mejora el rendimiento cuando el orden de los tokens es esencial para la tarea, pero si la tarea o el diseño del modelo ya capturan esas relaciones o si las secuencias no requieren esa información, su impacto puede ser limitado.